# Create Slice for Fabric Rack
This is the slice creation for the ```TrafficGeneration``` tutorial. This is the first step of the assignment: to create the slice on fabric using fablib
 
 

## 1. Set up the Experiment

In this section, you will use the Fablib manager to create a new slice that is composed of 2 nodes for the Traffic Generation Assignment.
### 1.1 Reserve Resources
In the 'EDUKY' site, we will reserve a set of 2 nodes arranged in a line, with one node designated as a server and the other as a client. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. The each node will have 1 network cards (NICs) to communicate with each other, We will call the server "Node_1" and the card "ND_A" and the client we will be "Node_2" and it's port "ND_B". The bridge connecting the two nodes is an l2network called "Lan". All nodes will be preloaded with an 'Ubuntu' Linux OS. Upon submission, the slice will be named 'TrafficGeneration'.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "TrafficGeneration"
site = "EDUKY"
print(f'site = {site}')

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    
    #first Node
    ND1 = slice.add_node(name="Node_1", site=site)
    ND1.set_capacities(cores=cores, ram=ram, disk=disk)
    ND1.set_image(image)
    Net_ND1 = ND1.add_component(model=nicmodel, name="ND_A").get_interfaces()[0]
    

    #Second Node
    ND2 = slice.add_node(name="Node_2", site=site)
    ND2.set_capacities(cores=cores, ram=ram, disk=disk)
    ND2.set_image(image)
    Net_ND2 = ND2.add_component(model=nicmodel, name="ND_B").get_interfaces()[0] 

    #Lan
    LN = slice.add_l2network(name="Lan", interfaces=[Net_ND1, Net_ND2])
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

### 1.2 Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs such that the nodes are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation stage so that we can provide the correct connection in the network, in this case, "Lan".

This slice contains a single subnetwork in IPv4: 10.0.0.0/24 (between server and the client, "Lan"). This slice will provide IP 10.0.0.1 to "Node_A" and 10.0.0.2 to "Node_B" following the guidelines set by the network subnet.

After providing the IPs to the corresponding devices, the cell will output the network status and execute "ip a" to check the connections that are active.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# configure nodes
try:
    A = slice.get_node('Node_1')
    B = slice.get_node('Node_2')
    
    subnet = IPv4Network("10.0.0.0/24")
    
    #A's lan
    A_iface = A.get_interface(network_name="Lan")  
    A_iface.ip_addr_add(addr='10.0.0.1', subnet=subnet)
    A_Device_Name = interface.get_device_name()
    #B's Lan
    B_iface = B.get_interface(network_name="Lan")  
    B_iface.ip_addr_add(addr='10.0.0.2', subnet=subnet)
    
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
            
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
        
except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
We only need to install a couple of tools for our slice, the first of these will be "nmap" and the second "iperf", these tools will help us monitor traffic inside of the nodes.

Additionally for this slice, we need to install some software that is only available from a GitHub repository. After we download the script from the GitHub repository, we modify some lines to execute according to our environment. The last step is to install the libraries that the scripts we downloaded from github have, this is the r-studio library: "r-base-core","r-cran-ggplot2" & "r-cran-littler" along with "moreutils". We also use system call to avoid saving necessary tcp data and chmod to turn one of the scripts we downloaded from github into an executable.

In [ ]:
# Install Software
try:
    INTERFACE_NAME_HOLDER = "INTERFACE_NAME"
    for node in slice.get_nodes():        
        node.execute("sudo apt update")   
        node.execute("sudo apt-get install nmap -y")
        node.execute("sudo apt-get install iperf -y")
        #Graphics
        node.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/assignments/TrafficGeneration/scripts/TG-data-analysis.R")
        node.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/assignments/TrafficGeneration/scripts/TG-output.sh")
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER}/{A_Device_Name}/' RT-output.sh")
        node.execute("sudo apt-get -y install moreutils r-base-core r-cran-ggplot2 r-cran-littler;sudo sysctl -w net.ipv4.tcp_no_metrics_save=1;chmod +x TG-output.sh ")
except Exception as e:
    print(f"Fail: {e}")

    

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the Traffic Generation notebook. You can either open it on the explorer or click [Here](./TrafficGeneration.ipynb) to open the next notebook.